In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
include("9msctrl.jl")
include("Magnitude Rand.jl")
include("Objective function Monitor.jl")
using PyCall
plt = pyimport("matplotlib.pyplot")

┌ Info: Recompiling stale cache file C:\Users\Kaijie\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


PyObject <module 'matplotlib.pyplot' from 'C:\\Users\\Kaijie\\.julia\\conda\\3\\lib\\site-packages\\matplotlib\\pyplot.py'>

In [2]:
dadt = zeros(9)
q    = zeros(9)
dqdt = zeros(9)
store = RAMStorage(zeros(9))
B = [0,0,0,0,1,0,0,0,0]


objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
function objfun(t, u, dudt, I, dIdt)
    return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
end
function objfun(xq::Coupled)
    return objfun(xq[1])
end


function minus_energy_grad(t, a, dadt, v, dvdt)
    dvdt[1] += -2*(a[1] - 1)
    for i = 2:9
        dvdt[i] += -2*a[i]
    end
    return dvdt
end

function Continuous_CTRL_Objfun(Reynolds,Initial_Time, Final_Time,uinitial,n,αv)
    u0 = uinitial
    Re = Reynolds
    Tf = Final_Time
    T = Tf/n
    Tfn = T
    Ti = Initial_Time
    ts = Ti:timestep:Tfn
    output_CTRL = zeros(1)
    α = αv
    
    for i in 1:n
        
        
        
        function Objective_fun_CTRL(η::AbstractVector) ##Take note on Re, T, and u0
            f = NineModeSystemCTRL(Re, ts, η)
            ϕ = flow(couple(f,objfun),RK4(couple(zeros(9),zeros(1))), TimeStepConstant(timestep))
            I0 = Float64[0.0]
            u0_local = copy(u0)
            ϕ(couple(u0_local, I0), extrema(ts))
            return I0[1]/T + α.*η2int(η, timestep) ## T is a constant
        end
        function Objective_fun_Gradient!(G::AbstractVector,η::AbstractVector)
            f = NineModeSystemCTRL(Re, ts, η)
            ϕ = flow(f,RK4(zeros(9)), TimeStepConstant(timestep))
            u0_local = copy(u0)
            ϕ(u0_local, (Ti, Tfn), reset!(store))

            h = NineModeSystemLin(Re, true, minus_energy_grad)
            ψ_adj = flow(h, RK4(zeros(9), ContinuousMode(true)), TimeStepFromStorage(timestep))
            mon = Monitor(zeros(9), q -> -B[5]*q[5])
            ψ_adj(zeros(9), store, (Tfn, Ti), reset!(mon))
            G .= samples(mon)[end:-1:1] .+ 2.0.*α.*η
        end
        function finalpoint(η)
            u0_local = copy(u0)
            f = NineModeSystemCTRL(Re, ts, η)
            F = flow(f, RK4(zeros(9)), TimeStepConstant(timestep))
            return F(u0_local, extrema(ts))
            end #functions which will be reiterated
        
        
        
        
        result = optimize(Objective_fun_CTRL, Objective_fun_Gradient!, zeros(length(ts)),LBFGS(),Optim.Options(f_tol = 1e-6,
                             store_trace = false,
                             show_trace = false))
        η = result.minimizer
        output_CTRL = vcat(output_CTRL, result.minimizer)
        u0 = finalpoint(η)
        
        Ti = Tfn + timestep
        Tfn = Tfn + T
        ts = Ti:timestep: Tfn
    end
    return output_CTRL[2:end]
end

Continuous_CTRL_Objfun (generic function with 1 method)

In [3]:
n = 10
timestep = 0.5
T1 = 200
Reynold_Number = 300
α = 0.1
m = 5
Tinterval = 1000
T2 = T1+Tinterval
ts = T1:timestep:T2
x2_controlled = 0;

In [4]:
Ini_rand = [] #Initial condition
for i in 1:n
    Ini_rand = push!(Ini_rand, mgscale(rand(9),0.3) + [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]) # Radius = 0.3
end

In [5]:
Second_Point = []
Uncontrolled_State = []
x = []

f = NineModeSystem(Reynold_Number)
ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestep))
for i in 1:length(Ini_rand)
    I0 = Float64[0.0]
    u0_local = copy(Ini_rand[i])
    mon = Monitor(couple(zeros(9), zeros(1)), objfun)
    a,b = ϕ(couple(u0_local, I0), (0, T1), reset!(mon))
    Uncontrolled_State = push!(Uncontrolled_State, samples(mon))
    x = times(mon)
    Second_Point = push!(Second_Point, a)
end

In [6]:
f = NineModeSystem(Reynold_Number)
ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestep))

for i in 1:length(Second_Point)
    u0 = copy(Second_Point[i])
    final = Continuous_CTRL_Objfun(Reynold_Number, T1, T2, u0, m, α)
    ff = NineModeSystemCTRL(Reynold_Number, ts, final)
    F = flow(couple(ff, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestep))
    I0 = Float64[0.0]
    u0_local = copy(u0)
    mon = Monitor(couple(zeros(9), zeros(1)), objfun)
    F(couple(u0_local,I0), extrema(ts), reset!(mon))
    Uncontrolled_State[i] = vcat(Uncontrolled_State[i],samples(mon)[2:end])
    x2_controlled = times(mon)[2:end]
end


x = vcat(x,x2_controlled);

In [7]:
fig,ax = plt.subplots()
for i in 1:length(Ini_rand)    
    ax.plot(x,Uncontrolled_State[i],color="grey", linestyle="dashed",linewidth = 0.1)
end

Main_line = (1/n).*sum(Uncontrolled_State)
ax.plot(x,Main_line,color="black",linewidth = 0.5,label = "α = $α")
ax.plot([200,200],[0,2.5],color="black", linestyle="--", linewidth = 0.5)
ax.legend() #Control state for one fixed alpha

PyObject <matplotlib.legend.Legend object at 0x00000000386C1080>

In [8]:
n = 4
timestep = 0.5
T1 = 200
Reynold_Number = 300
m = 5
T2 = T1+1000
ts = T1:timestep:T2
x2_controlled = 0
α_range = 0.01:0.01:0.15;

In [9]:
Ini_rand = []
for i in 1:n
    Ini_rand = push!(Ini_rand, mgscale(rand(9),0.2) + [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]) # Radius = 0.2
end

In [10]:
Full_State = []
Mean_State = []
Uncontrolled_State = []
x1_array = []
x2_array = []

Second_Point = []
for i in 1:length(Ini_rand)
    I0 = Float64[0.0]
    u0_local = copy(Ini_rand[i])
    mon = Monitor(couple(zeros(9), zeros(1)), objfun)
    a,b = ϕ(couple(u0_local, I0), (0, T1), reset!(mon))
    Uncontrolled_State = push!(Uncontrolled_State, samples(mon))
    x1_array = times(mon)
    Second_Point = push!(Second_Point, a)
end

for j in 1:length(α_range)
    α = α_range[j]
    f = NineModeSystem(Reynold_Number)
    ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestep))
    mean = 0
    for i in 1:length(Second_Point)
        u0 = copy(Second_Point[i])
        final = Continuous_CTRL_Objfun(Reynold_Number, T1, T2, u0, m, α)
        ff = NineModeSystemCTRL(Reynold_Number, ts, final)
        F = flow(couple(ff, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestep))
        I0 = Float64[0.0]
        u0_local = copy(u0)
        mon = Monitor(couple(zeros(9), zeros(1)), objfun)
        F(couple(u0_local,I0), extrema(ts), reset!(mon))
        Full_State[i] = push!(Full_State,samples(mon)[2:end])
        x2_array = times(mon)[2:end]
        mean = mean .+ samples(mon)
    end
    println("pass")
    Mean_State = push!(Mean_State, (1/n).*mean)
end

x_full_array = vcat(x1_array,x2_array)
fig,ax = plt.subplots()
for i in 1:length(Uncontrolled_State)
    ax.plot(x1_array,Uncontrolled_State[i],color="grey", linestyle="dashed",linewidth = 0.1)
end
ax.plot(x1_array,(1/n).*sum(Uncontrolled_State), color="black", linewidth = 0.5)

pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass


1-element Array{PyObject,1}:
 PyObject <matplotlib.lines.Line2D object at 0x0000000039C3FC50>

In [11]:
fig,ax = plt.subplots()

for i in 1:length(Uncontrolled_State)
    ax.plot(x1_array,Uncontrolled_State[i],color="grey", linestyle="dashed",linewidth = 0.1)
end

ax.plot(x1_array,(1/n).*sum(Uncontrolled_State), color="black", linewidth = 0.5)

# for i in 1:length(Full_State)
#     ax.plot(x2_array, Full_State[i], color="grey", linestyle="dashed",linewidth = 0.01)
# end

for i in 1:length(Mean_State)
    a = α_range[i]
    ax.plot(x2_array, Mean_State[i][2:end], linewidth = 0.9, label="α = $a")
end

ax.legend()
plt.xlabel("t")
plt.ylabel("J(t)")

PyObject Text(42.722222222222214, 0.5, 'J(t)')